In [ ]:
# !pip install transformers

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    ROBERTA_PATH = 'roberta-large'
    TOKENIZER_PATH = 'roberta-large'
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'roberta-large'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [9]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [10]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [11]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [12]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [13]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [14]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [15]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [16]:
class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.ROBERTA_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.ROBERTA_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [17]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [120]:
import re

for i, (name, param) in enumerate(sample_model.named_parameters()):
    if(name.find('layer') > -1):
        layer_name = re.sub(r'.+(layer\.\d+).+', r'\1', name)
        

layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.0
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.1
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.2
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.3
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.4
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.5
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.6
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7
layer.7


In [18]:
for i, (name, param) in enumerate(sample_model.named_parameters()):
    print(i, name, param.size())

0 transformer_model.embeddings.word_embeddings.weight torch.Size([50265, 1024])
1 transformer_model.embeddings.position_embeddings.weight torch.Size([514, 1024])
2 transformer_model.embeddings.token_type_embeddings.weight torch.Size([1, 1024])
3 transformer_model.embeddings.LayerNorm.weight torch.Size([1024])
4 transformer_model.embeddings.LayerNorm.bias torch.Size([1024])
5 transformer_model.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
6 transformer_model.encoder.layer.0.attention.self.query.bias torch.Size([1024])
7 transformer_model.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024])
8 transformer_model.encoder.layer.0.attention.self.key.bias torch.Size([1024])
9 transformer_model.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024])
10 transformer_model.encoder.layer.0.attention.self.value.bias torch.Size([1024])
11 transformer_model.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
12 transformer_model.enco

In [19]:
sample_input_ids = torch.randint(0, 1000, [8, 248])
sample_attention_mask = torch.randint(0, 1000, [8, 248])

In [20]:
sample_model(sample_input_ids, sample_attention_mask)[1].shape

torch.Size([8, 1024])

In [21]:
torch.sum(torch.randn([8, 496, 768]), axis=1)

tensor([[ 21.3262, -30.2247,  24.8986,  ...,  24.9654,  23.2924,  -4.5589],
        [  6.9908,   8.5307, -32.1037,  ...,   4.8191, -15.7707, -10.2882],
        [-32.8582, -26.4306,  17.7165,  ...,   7.1739, -13.1200,   9.3983],
        ...,
        [-22.0663, -18.5671,  50.3724,  ...,  24.3221,  39.5540,  54.6590],
        [-18.5394, -20.1138,   5.7116,  ..., -21.3238,   2.1945,  17.7358],
        [ 40.4799,  12.8752,  29.5825,  ..., -25.8740, -13.7510, -19.0250]])

### Evaluation and Prediction

In [32]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [33]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred, _ = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [34]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [115]:
5e-5 / 2.5, 5e-5 / 0.5, 5e-5

(2e-05, 0.0001, 5e-05)

In [35]:
def create_optimizer(model, base_lr=5e-5):
    named_parameters = list(model.named_parameters())
    
    regressor_param_start = 395
    roberta_parameters = named_parameters[:389]
    attention_parameters = named_parameters[391:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5 # 2e-05
        if layer_num >= 260:
            lr = base_lr / 0.5 # 1e-4
        elif layer_num >= 132:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [36]:
sample_optimizer = create_optimizer(sample_model)

In [37]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [38]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [39]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [48]:
class Trainer():
    def __init__(self, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                pred, _ = self.model(input_ids, attention_mask)
                
                mse = mse_loss(pred.flatten(), target)
                
                mse.backward()
                
                self.optimizer.step()
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, self.model_path)
                    start = time.time()
                
                step += 1
        return best_val_rmse

In [49]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = kfold.split(train_df)

In [50]:
list_val_rmse = []

pbar = tqdm(enumerate(splits), total=cfg.NUM_FOLDS, position=0, leave=True)
for fold, (train_indices, val_indices) in pbar:
    pbar.set_description(f'Fold {fold}')
    model_path = cfg.MODEL_FOLDER/f"{cfg.model_name.replace('/', '_')}_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=True, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
        
    optimizer = create_optimizer(model)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    trainer = Trainer(model, model_path, train_loader, val_loader, optimizer, scheduler = scheduler)
    list_val_rmse.append(trainer.train())
    
    tokenizer.save_pretrained(str(model_path.parent))
    del model
    gc.collect()
    
    if cfg.DEVICE == 'cuda':
        torch.cuda.empty_cache()
        
print("\nPerformance estimates:")
print(list_val_rmse)
print("Mean:", np.array(list_val_rmse).mean())

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 13.1 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.875 New best_val_rmse: 0.875

16 steps took 11.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.8035 New best_val_rmse: 0.8035

16 steps took 11.9 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6572 New best_val_rmse: 0.6572

16 steps took 12.0 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6746 Still best_val_rmse: 0.6572 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6389 New best_val_rmse: 0.6389

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5795 New best_val_rmse: 0.5795

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6338 Still best_val_rmse: 0.5795 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5588 New best_val_rmse: 0.5588

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.7001 Still best_val_rmse: 0.5588 (from epoch 0)

16 steps took 12.4 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5681 Still best_val_rmse: 0.5588 (from 

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 13.4 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.951 New best_val_rmse: 0.951

16 steps took 12.1 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7727 New best_val_rmse: 0.7727

16 steps took 12.1 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.9526 Still best_val_rmse: 0.7727 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6551 New best_val_rmse: 0.6551

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7483 Still best_val_rmse: 0.6551 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.554 New best_val_rmse: 0.554

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5987 Still best_val_rmse: 0.554 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5535 New best_val_rmse: 0.5535

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5732 Still best_val_rmse: 0.5535 (from epoch 0)

16 steps took 12.4 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5535 Still best_val_rmse:

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...


16 steps took 13.4 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.981 New best_val_rmse: 0.981

16 steps took 12.1 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.725 New best_val_rmse: 0.725

16 steps took 12.1 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7262 Still best_val_rmse: 0.725 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.8067 Still best_val_rmse: 0.725 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6729 New best_val_rmse: 0.6729

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5901 New best_val_rmse: 0.5901

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5784 New best_val_rmse: 0.5784

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5851 Still best_val_rmse: 0.5784 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6704 Still best_val_rmse: 0.5784 (from epoch 0)

16 steps took 12.3 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.533 New best_val_rmse: 0.5

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 13.4 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.021 New best_val_rmse: 1.021

16 steps took 12.0 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7651 New best_val_rmse: 0.7651

16 steps took 12.0 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.8746 Still best_val_rmse: 0.7651 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7095 New best_val_rmse: 0.7095

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6394 New best_val_rmse: 0.6394

16 steps took 12.0 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6128 New best_val_rmse: 0.6128

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5775 New best_val_rmse: 0.5775

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.548 New best_val_rmse: 0.548

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.4921 New best_val_rmse: 0.4921

8 steps took 6.35 seconds
Epoch: 1 batch_num: 4 val_rmse: 0.4865 New best_val_rmse: 0.4865

4 steps took 3.02 seconds
Epoch: 1 batch_num:

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



16 steps took 13.4 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9516 New best_val_rmse: 0.9516

16 steps took 12.1 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.021 Still best_val_rmse: 0.9516 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7332 New best_val_rmse: 0.7332

16 steps took 12.1 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6273 New best_val_rmse: 0.6273

16 steps took 12.1 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.621 New best_val_rmse: 0.621

16 steps took 12.1 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5901 New best_val_rmse: 0.5901

16 steps took 12.1 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.7474 Still best_val_rmse: 0.5901 (from epoch 0)

16 steps took 12.1 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5706 New best_val_rmse: 0.5706

16 steps took 12.1 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5412 New best_val_rmse: 0.5412

16 steps took 12.4 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5421 Still best_val_rmse: 0.5412 (from epoch 0)

16 steps

### Verify the model

In [51]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [52]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [53]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [54]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model 2


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model 3


...

Model 5


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model 6


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU times: user 21.4 s, sys: 6.22 s, total: 27.7 s
Wall time: 34.1 s


In [60]:
from transformers import RobertaTokenizer

tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = RobertaTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}")
    tokenizers.append(tokenizer)

In [61]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [62]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [63]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [64]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [65]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [66]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [67]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.29372574368263743
Fold 1 (2361,) (472,)
rmse_score 0.2741654981826458
Fold 2 (2361,) (472,)
rmse_score 0.2576478336983424
Fold 3 (2361,) (472,)
rmse_score 0.23302212661130733
Fold 4 (2361,) (472,)
rmse_score 0.2769448188602186
Fold 5 (2361,) (472,)
rmse_score 0.27798433526183103
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3065460871822688
Fold 1 (2361,) (472,)
rmse_score 0.3170843648826062
Fold 2 (2361,) (472,)
rmse_score 0.3054054074203651
Fold 3 (2361,) (472,)
rmse_score 0.3119457081396984
Fold 4 (2361,) (472,)
rmse_score 0.31732000188468534
Fold 5 (2361,) (472,)
rmse_score 0.3257952442958126
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.2662407951144236
Fold 1 (2361,) (472,)
rmse_score 0.24997115087718932
Fold 2 (2361,) (472,)
rmse_score 0.23977221848803673
Fold 3 (2361,) (472,)
rmse_score 0.26245030395837665
Fold 4 (2361,) (472,)
rmse_score 0.2544690001034037
Fold 5 (2361,) (472,)
rmse_score 0.2707506432777301
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.28128696839796347
Fold 1 (2361,) (472,)
rmse_score 0.2898404386597716
Fold 2 (2361,) (472,)
rmse_score 0.27412481504699376
Fold 3 (2361,) (472,)
rmse_score 0.2628630065736049
Fold 4 (2361,) (472,)
rmse_score 0.2672996276087227
Fold 5 (2361,) (472,)
rmse_score 0.27282195828480155
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.27695075853621476
Fold 1 (2361,) (472,)
rmse_score 0.2757455604951166
Fold 2 (2361,) (472,)
rmse_score 0.2529282218860511
Fold 3 (2361,) (472,)
rmse_score 0.2710761463721579
Fold 4 (2361,) (472,)
rmse_score 0.2907050707380049
Fold 5 (2361,) (472,)
rmse_score 0.29952987921589025
FINAL RMSE score 0.27854712445789576
CPU times: user 4min 48s, sys: 3.81 s, total: 4min 52s
Wall time: 4min 51s


In [68]:
final_kernel_predictions_means

[[0.003269142824678698],
 [-0.0013352013265169353],
 [0.001247882678708451],
 [0.0010360796004914576],
 [0.0008872567149352553]]

In [69]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [70]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.20172898, 0.19363321, 0.20381828, 0.20068947, 0.20013005])

In [71]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [72]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.905523425553752)

In [73]:
final_scores_flat

array([-0.37529463, -0.37176504, -0.44579597, -2.32708045, -1.90310749,
       -1.21147066,  0.29975715])

In [74]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(-0.05469209360234095, -0.01093841872046819)

In [75]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.429987
1,f0953f0a5,-0.426457
2,0df072751,-0.500488
3,04caf4e0c,-2.381773
4,0e63f8bea,-1.957800
5,12537fe78,-1.266163
6,965e592c0,0.245065


### Prepare Packaging

In [76]:
cfg.model_name

'roberta-large'

In [100]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [101]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/roberta-large/best')

In [102]:
cfg.NUM_FOLDS

6

In [103]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [104]:
bestmodels

[PosixPath('/home/commonlit/models/roberta-large_1'),
 PosixPath('/home/commonlit/models/roberta-large_2'),
 PosixPath('/home/commonlit/models/roberta-large_3'),
 PosixPath('/home/commonlit/models/roberta-large_4'),
 PosixPath('/home/commonlit/models/roberta-large_5'),
 PosixPath('/home/commonlit/models/roberta-large_6')]

In [105]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    i = i + 1
    best_model_file = f'{best_model}/model_{i}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/tokenizer_config.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/vocab.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        merges = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}_{i}/merges.txt'))
        assert merges.exists()
        copyfile(merges, tokenizer_path/'merges.txt')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [106]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/roberta-large/best_models.zip'

In [107]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip  dataset-metadata.json  lm.zip  roberta-large.yaml


In [108]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/roberta-large.yaml': No such file or directory


In [109]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [110]:
!du -h {MODELS_PATH/cfg.model_name}/*

1.3M	/home/commonlit/models/roberta-large/best/tokenizer-1
1.3M	/home/commonlit/models/roberta-large/best/tokenizer-2
1.3M	/home/commonlit/models/roberta-large/best/tokenizer-3
1.3M	/home/commonlit/models/roberta-large/best/tokenizer-4
1.3M	/home/commonlit/models/roberta-large/best/tokenizer-5
1.3M	/home/commonlit/models/roberta-large/best/tokenizer-6
8.0G	/home/commonlit/models/roberta-large/best
7.3G	/home/commonlit/models/roberta-large/best_models.zip
4.0K	/home/commonlit/models/roberta-large/dataset-metadata.json
1.4G	/home/commonlit/models/roberta-large/lm
1.3G	/home/commonlit/models/roberta-large/lm.zip
4.0K	/home/commonlit/models/roberta-large/roberta-large.yaml


In [111]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/roberta-large/lm.zip'

In [93]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/roberta-large/dataset-metadata.json


In [94]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [95]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [96]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}-light').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}-light')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-roberta-large-light",
  "id": "gilfernandes/commonlit-roberta-large-light",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [97]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [98]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 8.45G/8.45G [14:14<00:00, 10.6MB/s]
Upload successful: best_models.zip (8GB)
Starting upload for file roberta-large.yaml
100%|███████████████████████████████████████████| 115/115 [00:08<00:00, 14.3B/s]
Upload successful: roberta-large.yaml (115B)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 1.20G/1.20G [02:01<00:00, 10.6MB/s]
Upload successful: lm.zip (1GB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-roberta-large-light


In [112]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with merges.txt" -d

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 7.24G/7.24G [12:39<00:00, 10.2MB/s]
Upload successful: best_models.zip (7GB)
Starting upload for file roberta-large.yaml
100%|███████████████████████████████████████████| 115/115 [00:08<00:00, 14.4B/s]
Upload successful: roberta-large.yaml (115B)
Starting upload for file lm.zip
100%|██████████████████████████████████████| 1.20G/1.20G [02:35<00:00, 8.31MB/s]
Upload successful: lm.zip (1GB)
Skipping folder: best; use '--dir-mode' to upload folders
Skipping folder: lm; use '--dir-mode' to upload folders
Dataset version is being created. Please check progress at /api/v1/datasets/status/gilfernandes/commonlit-roberta-large-light


In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)